<a href="https://colab.research.google.com/github/koh-ts/traffic_volume_prediction/blob/master/Copy_of_traffic_volume_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
import pandas as pd
import datetime
import pandas_profiling as pdp
from IPython.display import HTML
import xgboost as xgb
import lightgbm as lgb

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
root_dir = '/content/drive/My Drive/report1/'

N = 30000
nfeat = 8

train_data = pd.read_csv(root_dir + 'train.csv')
test_data = pd.read_csv(root_dir + 'test.csv')

Xmat = np.matrix(train_data)[:,:nfeat]
Ymat = np.matrix(train_data)[:,nfeat]
# Ymat = np.matrix(train_data['traffic_volume'])

In [4]:
all_data = pd.concat([train_data, test_data], ignore_index=True)
all_data = all_data[['timestamp', 'holiday', 'temperature', 'weather', 'weather_detail', 'rain_in_hour', 'snow_in_hour', 'clouds_cover', 'traffic_volume']]
# all_data.head(4)
# all_data[['holiday'][:2]]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
# profile1 = pdp.ProfileReport(all_data)
# profile1.to_file(outputfile="myoutputfile1.html")
# HTML(filename='myoutputfile1.html')

In [0]:
# lis = [[0,0] for _ in range(24)]
# print(lis)
# for index, d in all_data.iterrows():
#   if index == len(train_data):
#     break
#   n = int(d['timestamp'][11:13])
#   lis[n][1] += 1
#   lis[n][0] += d['traffic_volume']
# mean = []
# for item in lis:
#   mean.append(item[0] / item[1])
# print(mean)

In [0]:
mean_traffic = [844.1971481711097, 520.6844720496895, 391.89875, 372.22609246358456, 701.162890385793, 2084.489069331668, 4167.179251074279, 4742.792975970425, 4608.443757725587, 4386.964470284238, 4189.753390875462, 4475.000661375661, 4729.360078277887, 4728.403106009453, 4949.089494163424, 5252.46875, 5675.80192926045, 5341.854208084825, 4283.009658725048, 3287.383257030739, 2841.853092783505, 2688.3029525032093, 2215.9378566899177, 1472.6247678018576]

In [0]:
# cnt = 0
for index, d in all_data.iterrows():
  # cnt += 1
  y = int(d['timestamp'][0:4])
  m = int(d['timestamp'][5:7])
  dd = int(d['timestamp'][8:10])
  n = int(d['timestamp'][11:13])
  if all_data.iat[index - n, 1] != 'None':
    all_data.at[index, 'isholiday'] = 1
  else:
    all_data.at[index, 'isholiday'] = 0
  # all_data.at[index, 'time'] = mean_traffic[n]
  all_data.at[index, 'time'] = n
  dayy = datetime.date(y, m, dd).weekday()
  all_data.at[index, 'day'] = dayy



  # if index == len(train_data):
  #   print(d)
  # if cnt == 100:
  #   break

In [0]:
all_data['weather_detail'] = all_data['weather_detail'].str.replace('Sky is clear', 'clear')
all_data['weather_detail'] = all_data['weather_detail'].str.replace('sky is clear', 'clear')
all_data['weather_detail'] = all_data['weather_detail'].str.replace('Sky is Clear', 'clear')
all_data['temperature'] = all_data['temperature'].replace(0.0, all_data['temperature'].median())

In [0]:
# all_data[all_data['holiday'] == "Martin Luther King Jr Day"]

In [0]:
meta_t = pd.get_dummies(all_data['time'])
meta_w = pd.get_dummies(all_data['weather'])
meta_wd = pd.get_dummies(all_data['weather_detail'])

In [12]:
profile2 = pdp.ProfileReport(all_data)
profile2.to_file(outputfile="myoutputfile2.html")
HTML(filename='myoutputfile2.html')

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,12
Number of observations,45645
Total Missing (%),1.5%
Total size in memory,4.2 MiB
Average record size in memory,96.0 B
Numeric,7
Categorical,4
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [0]:
del all_data['timestamp']
del all_data['weather']
del all_data['holiday']
del all_data['weather_detail']

In [0]:
all_data = all_data[['temperature', 'rain_in_hour', 'snow_in_hour', 'clouds_cover', 'time', 'isholiday', 'day', 'traffic_volume']]
all_data.columns
l = len(all_data.columns) - 1
# all_data.head(3)

In [0]:
X = np.float64(np.hstack([meta_w.iloc[:len(train_data),:],meta_wd.iloc[:len(train_data),:],meta_t.iloc[:len(train_data),:],all_data.iloc[:len(train_data),:l]]))
Y = np.float64(Ymat)

# XTrain = X[:N,:]
# YTrain = Y[:N,:]
# XVal = X[N:,:]
# YVal = Y[N:,:]

XTrain, XVal, YTrain, YVal = train_test_split(X, Y, test_size = 0.2, random_state = 66)

In [0]:
def RMSE(vec1, vec2):
  return np.sqrt(np.mean(np.power(vec1-vec2,2)))

In [0]:
reg = Ridge(alpha=100).fit(XTrain, YTrain)

In [18]:
yHatTrain = reg.predict(XTrain)
yHatVal = reg.predict(XVal)
score_reg = cross_val_score(reg, X, Y, cv=5, n_jobs=-1)
print(score_reg)
print(score_reg.mean())

print("Training error ", RMSE(YTrain, yHatTrain))
print("Validation error ", RMSE(YVal, yHatVal))

[ 0.80084493  0.79857174 -2.524439    0.78062144  0.81336293]
0.1337924070949792
Training error  883.2140988260348
Validation error  4850.839344159874


In [0]:
dtrain = xgb.DMatrix(XTrain, label=YTrain)
dvalid = xgb.DMatrix(XVal, label=YVal)
num_round = 5000
evallist = [(dvalid, 'eval'), (dtrain, 'train')]
evals_result = {}
param = {'max_depth': 3, 'eta': 0.01, 'objective': 'reg:squarederror',}
bst = xgb.train(param, dtrain, num_round, evallist, evals_result=evals_result, early_stopping_rounds=1000)

In [0]:
XTest = np.float64(np.hstack([meta_w.iloc[len(train_data):,:],meta_wd.iloc[len(train_data):,:],meta_t.iloc[len(train_data):,:],all_data.iloc[len(train_data):,:l]]))

In [0]:
dtest = xgb.DMatrix(XTest)
yHatTest_xgb = bst.predict(dtest)
np.reshape(yHatTest_xgb, (len(yHatTest_xgb), 1))
np.savetxt(root_dir+'result_xgb.txt', yHatTest_xgb)

In [0]:
yHatTest = reg.predict(XTest)
np.savetxt(root_dir+'result.txt', yHatTest) #save predictions in rows

In [0]:
res = []
for i in YTrain:
  d = float(i[0])
  res.append(d)
resv = []
for i in YVal:
  d = float(i[0])
  resv.append(d)

In [0]:
train_data_set = lgb.Dataset(XTrain, res)
valid_data_set = lgb.Dataset(XVal, resv, reference=train_data_set)

params = {'boosting_type': 'gbdt',
          'objective': 'regression',
          'metric': {'rmse'},
          'num_leaves': 32,
          'learning_rate': 0.05,
          'feature_fraction': 0.9,
          'bagging_fraction': 0.6, #0.8
          'bagging_freq': 5,
          'lambda_l2': 2}

In [50]:
gbm = lgb.train(params, train_data_set, num_boost_round=200, valid_sets=[train_data_set, valid_data_set], early_stopping_rounds=10)
yHatTest_gbm = gbm.predict(XTest)
np.reshape(yHatTest_gbm, (len(yHatTest_gbm), 1))

[1]	training's rmse: 1902.23	valid_1's rmse: 1901.72
Training until validation scores don't improve for 10 rounds.
[2]	training's rmse: 1814.73	valid_1's rmse: 1813.88
[3]	training's rmse: 1745.25	valid_1's rmse: 1744.2
[4]	training's rmse: 1666.07	valid_1's rmse: 1664.85
[5]	training's rmse: 1591.17	valid_1's rmse: 1589.75
[6]	training's rmse: 1520.52	valid_1's rmse: 1518.87
[7]	training's rmse: 1453.72	valid_1's rmse: 1451.82
[8]	training's rmse: 1390.65	valid_1's rmse: 1388.59
[9]	training's rmse: 1330.96	valid_1's rmse: 1328.71
[10]	training's rmse: 1274.6	valid_1's rmse: 1272.24
[11]	training's rmse: 1221.68	valid_1's rmse: 1219.21
[12]	training's rmse: 1174.39	valid_1's rmse: 1171.98
[13]	training's rmse: 1127.13	valid_1's rmse: 1124.56
[14]	training's rmse: 1083.16	valid_1's rmse: 1080.62
[15]	training's rmse: 1041.38	valid_1's rmse: 1038.86
[16]	training's rmse: 1014.76	valid_1's rmse: 1012.27
[17]	training's rmse: 977.166	valid_1's rmse: 974.57
[18]	training's rmse: 941.922	va

array([[ 635.05968602],
       [ 395.60361097],
       [ 346.21212771],
       ...,
       [2386.96322046],
       [1829.20229879],
       [1299.24792301]])

In [0]:
np.savetxt(root_dir+'result_lgbm.txt', yHatTest_gbm)

In [0]:
result = []
for xgbb, lgbm in zip(yHatTest_xgb, yHatTest_gbm):
  result.append(xgbb * 0.7 + lgbm * 0.3)
np.savetxt(root_dir+'result_ans.txt', res)